# Question word prediction

> Group 12: Tristan Perrot & Romain Darous

Task is to train and evaluate a **char per char Transformer model** model using any available QA-corpus, for instance, the [SQuAD corpus](https://rajpurkar.github.io/SQuAD-explorer/).


METTRE EN CONTEXTE LE DEBUT DE LA QUESTION EN CORRIGEANT LE CODE DEJA FAIT
ADAPTER POUR AUGMENTER LE NOMBRE DE TRANSFORMERS EVENTUELLEMENT ET FAIRE PLUSIEURS COUCHES

# 0. Importing modules

In [7]:
import json
import math
import os

# Importing
import matplotlib.pyplot as plt
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange
import random
from datetime import datetime
import numpy as np

# Importing models
import char_dataset
import cpc_model


c:\Users\romai\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# 1. Data pre-processing

## 1.1. Loading the dataset
**Note :** we only want to be able te recover the beginning of a question. For that, it doesn"t matter whether the question is impossible to answer or not.

In [8]:
data_dir = 'data'
if data_dir not in os.listdir():
    os.mkdir(data_dir)

if "squad_train.json" not in os.listdir(data_dir):
    # Download data at https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
    res = requests.get(
        "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
    data = json.loads(res.text)

    # Save data to file
    with open(data_dir + "/squad_train.json", "w") as f:
        json.dump(data, f)

with open(data_dir + "/squad_train.json", "r") as f:
    data = json.load(f)

# Extract answer text and question text
answers = []
questions = []
for article in data["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            
            """if qa["is_impossible"] :
                continue
            answers.append(qa["answers"][0]["text"])"""
                
            if qa["is_impossible"] and len(qa["question"]) > 4:
                continue
            else :
                answers.append("")
            questions.append(qa["question"])
            

print("Number of questions:", len(questions))

# Print some examples
for i in range(5):
    print()
    print("Question:", questions[i])
    print("Answer:", answers[i])

Number of questions: 86821

Question: When did Beyonce start becoming popular?
Answer: 

Question: What areas did Beyonce compete in when she was growing up?
Answer: 

Question: When did Beyonce leave Destiny's Child and become a solo singer?
Answer: 

Question: In what city and state did Beyonce  grow up? 
Answer: 

Question: In which decade did Beyonce become famous?
Answer: 


In [9]:
print(questions[i] + ' ' + answers[i])

In which decade did Beyonce become famous? 


## 1.2. Making a suitable dataset
``<BOS>`` token. Indicates that the sentence is starting.

We will make the prediction of the sentence in reverse mode, as we want to predict the beginning of a question. We will use unidirectionnal attention as well.

In [10]:
# Concatenating questions and answers
dataset = [(questions[i].lower() + ' ' + answers[i].lower())[::-1] for i in range(len(questions))]

# Shuffle dataset
random.shuffle(dataset)

# Splitting into train, validation, and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size = int(0.1*len(dataset)), train_size=int(0.9*len(dataset)))
train_dataset, val_dataset = train_test_split(train_dataset, train_size=int(0.85*len(train_dataset)), test_size = int(0.15*len(train_dataset)))

In [11]:
print(f"Size of the dataset : {len(dataset)}")
print(f"Size of the train, val and test sets : {len(train_dataset), len(val_dataset), len(test_dataset)}")
print(f"Example of original datapoint : {questions[0] + ' ' + answers[0]}")
print(f"Example of formatted datapoint : {dataset[0]}")

Size of the dataset : 86821
Size of the train, val and test sets : (66417, 11720, 8682)
Example of original datapoint : When did Beyonce start becoming popular? 
Example of formatted datapoint :  ?lavirra naeporue fo emit eht yb enog erew taht snoitazilivic eht fo emos tuoba wonk ew od woh


## 1.3. Building a character dataset
The dataset will be built in the ``train_charlm`` function as it depends on the desired configuration. The function that allow to build the dataset and their context are provided in the file ``char_dataset.py``.

# 2. The model

In [12]:
# Updating models# Delete the modules from the namespace
del char_dataset
del cpc_model

# Unload the modules from memory
import sys
del sys.modules['char_dataset']
del sys.modules['cpc_model']

# Importing models
import char_dataset
import cpc_model

## 2.0. Hyperparameters

In [13]:
# ============= Hyper-parameter class for training ============== #

class Config :
    def __init__(self, seq_ctxt = True, MAXLEN = 32) : 
        self.number_of_transformer_encoders = 4
        self.number_of_attention_heads = 4
        self.hidden_size = 256
        self.dropout_prob = 0.1
        self.batch_size = 64
        self.learning_rate = 0.0003
        self.weight_decay = 0.000001
        self.no_of_epochs = 0
        self.is_causal = True # When True, the attention is causal
        self.seq_ctxt = seq_ctxt # When False, forces the context to take the beginning of the answer into account
        self.MAXLEN = MAXLEN

## 2.1. Training
Defining functions for training and testing models.

In [14]:
def train_charlm(config, device) :
    start_time = datetime.now()

    # ==================== Building datasets ================ #
    train_char_set = char_dataset.CharDataset(train_dataset, config.MAXLEN, seq_ctxt=config.seq_ctxt)
    val_char_set = char_dataset.CharDataset(val_dataset, config.MAXLEN, seq_ctxt=config.seq_ctxt)

    # ======================= Training ======================= #

    

    training_loader = DataLoader(train_char_set, batch_size=config.batch_size)
    validation_loader = DataLoader(val_char_set, batch_size=config.batch_size)

    charlm = cpc_model.CharLM( config, len(char_dataset.CharDataset.id_to_char), config.MAXLEN, config.is_causal).to(device)
    criterion = nn.CrossEntropyLoss()
    charlm_optimizer = optim.Adam( charlm.parameters(), lr=config.learning_rate )


    patience = 3
    best_val_loss = float('inf')

    charlm.train()
    print( datetime.now().strftime("%X"), "Training starts" )

    iteration = 0
    for epoch in tqdm(range(config.no_of_epochs)) :
        val_loss = 0.0
        for input_tensor, label in training_loader :
            input_tensor, label = input_tensor.to(device), label.to(device)
            charlm_optimizer.zero_grad()
            logits = charlm(input_tensor).to(device)
            loss = criterion(logits.squeeze(1), label)
            loss.backward()
            charlm_optimizer.step()
        iteration += 1

        print( datetime.now().strftime("%X"), "End of epoch", epoch+1, ", loss=", loss.detach().item())
        
        # Validation phase with Early Stopping
        charlm.eval()
        
        with torch.no_grad():
            for input_tensor, label in validation_loader:
                input_tensor, label = input_tensor.to(device), label.to(device)
                logits = charlm(input_tensor).to(device)
                loss = criterion(logits.squeeze(1), label)
                val_loss += loss.item()

        val_loss /= len(validation_loader)
        print(datetime.now().strftime("%X"), "Validation loss=", val_loss)

        # Check early stopping condition
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            patience = 3
        else:
            patience -= 1

        if patience == 0:
            print("Early stopping at epoch", epoch + 1)
            break

        charlm.train()
    
    end_time = datetime.now()

    return charlm, iteration + 1, end_time - start_time


In [15]:
def test_charlm(model, config, device) :

    # Computing output on the test set
    test_char_set = char_dataset.CharDataset(test_dataset, config.MAXLEN, seq_ctxt=config.seq_ctxt)
    test_loader = DataLoader(test_char_set, batch_size=config.batch_size)

    accuracies = []
    model.to(device)
    model.eval()    
    with torch.no_grad():
        for input_tensor, label in test_loader:
            input_tensor, label = input_tensor.to(device), label.to(device)
            logits = model(input_tensor).to(device)
            _, pred_label = logits.topk(1, dim=-1)
            pred_label = pred_label.squeeze(-1)  # Squeeze to remove extra dimension
            accuracy = (pred_label == label).float().mean().item()  # Compute accuracy for this batch
            accuracies.append(accuracy)  # Append batch accuracy to list)

    total_acc = np.round(np.mean(np.array(accuracies))*100, 2)
    print(f"Test accuracy : {total_acc} %")
    return total_acc

## 2.2. Grid search on parameters

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print( "Running on", device )

Running on cpu


In [17]:
output = './output/'
model_names = ['charlm_seq_32_es', 'charlm_seq_64_es', 'charlm_nseq_32_es', 'charlm_nseq_64_es']
configs = [Config(seq_ctxt=True, MAXLEN=32), Config(seq_ctxt=True, MAXLEN=64),Config(seq_ctxt=False, MAXLEN=32), Config(seq_ctxt=False, MAXLEN=64)]
test_acc = []
models = []
train_time = []
epochs = []

for i, config in enumerate(configs) :
    # Training the model
    model, epoch, delta = train_charlm(config, device)
    models.append(model)
    train_time.append(delta)
    epochs.append(epoch)


16:34:14 Training starts


0it [00:00, ?it/s]

16:34:20 Training starts


0it [00:00, ?it/s]

16:34:26 Training starts


0it [00:00, ?it/s]

16:34:32 Training starts


0it [00:00, ?it/s]

In [19]:
models

[CharLM(
   (embed): Embedding(185, 256)
   (transformers): ModuleList(
     (0-3): 4 x EncoderBlock(
       (attn): MultiHeadSelfAttention(
         (query): Linear(in_features=256, out_features=256, bias=False)
         (key): Linear(in_features=256, out_features=256, bias=False)
         (value): Linear(in_features=256, out_features=256, bias=False)
         (final): Linear(in_features=256, out_features=256, bias=False)
       )
       (ffn): PositionwiseFFN(
         (fc1): Linear(in_features=256, out_features=256, bias=True)
         (fc2): Linear(in_features=256, out_features=256, bias=True)
         (dropout): Dropout(p=0.1, inplace=False)
       )
       (dropout): Dropout(p=0.1, inplace=False)
       (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
       (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
     )
   )
   (final): Linear(in_features=8192, out_features=185, bias=True)
 ),
 CharLM(
   (embed): Embedding(185, 256)
   (transformers): ModuleList

In [20]:
assert len(models) == len(configs)
for i, config in enumerate(configs) :
  accuracy = test_charlm(models[i], config, device)
  test_acc.append(accuracy)
  # Saving the model
  torch.save(models[i].state_dict(), f"{output}{model_names[i]}{epoch}.pth")
  print("Model saved successfully")

KeyboardInterrupt: 

Building a ``.csv`` file to store the results :

In [ ]:
import pandas as pd

data = {
    'Running time' : train_time,
    'Last epoch' : epochs,
    'Type of context' : ['Sequential' if cfg.seq_ctxt else 'Split' for cfg in configs],
    'Context size' : [cfg.MAXLEN for cfg in configs],
    'Test accuracy (%)' : test_acc
}

# Create the DataFrame with row names
df = pd.DataFrame(data, index=model_names)
df.to_csv(output + 'metrics.csv')

## 2.2. Loading a model
Now that we trained several models, we can load them.

In [ ]:
"""# Load a model
charlm = cpc_model.CharLM( config, len(char_dataset.CharDataset.id_to_char), config.MAXLEN, config.is_causal).to(device)
charlm.load_state_dict(torch.load('./output/charlm_seq_model_early_stopping.pth'))
charlm.eval()"""

# Getting the best model
best_model_idx = 3
charlm = models[best_model_idx]
config = configs[best_model_idx]
#print("Best model :", df.loc[model_names[best_model_idx]])

## 2.2. Evaluation on the best model

## 2.3. User interaction

In [ ]:
# ==================== User interaction ==================== #
while True:
    text = input("> ").strip()
    if text == "" :
        continue
    elif text == "QUIT" :
        break

    # Will be used to output question
    full_question = list(text.lower())[::-1]

    if full_question[-1] != ' ' : 
        full_question.append(' ')
    if '?' not in full_question :
        full_question = ['?'] + full_question
        
    new_character = full_question[-1]

    # Recovering the beginning of the question
    try :
        count = 0
        MAX_COUNT = 50
        while new_character != char_dataset.CharDataset.BOQ and count < MAX_COUNT :
            # Building context
            char_list = []
            if config.seq_ctxt :
                char_list = full_question[-config.MAXLEN:]
            else :
                tmp = "".join(full_question)
                words_a, words_q = tmp.split("?")[0], tmp.split("?")[1]
                len_q = len(words_q) # counting the <BOS> token
                if len_q <= config.MAXLEN//2 or len(words_a) < config.MAXLEN//8 :
                    char_list = full_question[-config.MAXLEN:]
                else :
                    char_list = words_a[config.MAXLEN//2:] + full_question[-config.MAXLEN:]
            
            ctxt = [char_dataset.CharDataset.char_to_id[c] if c in char_dataset.CharDataset.char_to_id 
                    else char_dataset.CharDataset.char_to_id[char_dataset.CharDataset.char_to_id[char_dataset.CharDataset.PADDING_SYMBOL]] 
                    for c in char_list]
            
            
            # Computing the next character
            input_tensor = torch.tensor( [0]*(config.MAXLEN-len(ctxt)) + ctxt).unsqueeze(0).to(device)
            logits = charlm(input_tensor).squeeze().to(device)
            _, new_character_tensor = logits.topk(1)
            new_character = char_dataset.CharDataset.id_to_char[new_character_tensor.detach().item()]
            
            # Uploading the final output
            full_question.append(new_character)
            count += 1

        full_question = "".join(full_question[::-1])
        print(f"Recovered question : {full_question}")
    except KeyError :
        print("ERROR")
        continue

>  are you ? Fine


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
